<a href="https://colab.research.google.com/github/jhmuller/mlp-covid19/blob/main/mlp_covid19_1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 1 - Loading and Cleaning


## changing the indent
Enter the snippet below into the browser's javascript console
on chrome get to that from "..."->more tools->developer tools->console
```
var cell = Jupyter.notebook.get_selected_cell();
var config = cell.config;
var patch = {
      CodeCell:{
        cm_config:{indentUnit:2}
      }
    }
config.update(patch)
```

In [ ]:
# need to intall this every time on colab, not sure why but I do it
import importlib
import warnings
import pdb
import datetime
now = datetime.datetime.now
def warning_on_one_line(message, category, filename, lineno, file=None, line=None):
    return 'line:%s  cat:<%s>  msg:%s\n'% (lineno, category.__name__, message)  

warnings.formatwarning = warning_on_one_line

def global_imports(imports=None, verbosity=0):
  fails = []
  if not imports:
    return
  if not isinstance(imports, list):
    msg = "imports shoule be a list with elements like\m"
    msg += "'foo' or 'foo as bar'"
    warnings.warn(msg)
    return
  #pdb.set_trace()
  for line in imports:
    parts = line.split()
    if len(parts) not in [1,3] or \
          (len(parts) == 3 and parts[1] != 'as'):
      msg = "<{0}> not a valid import line\n"
      msg += "use 'import foo' or 'import foo as bar'"
      warnings.warn(msg)
      continue
    module_name = parts[0]
    asname = parts[0]
    if len(parts) == 3:
      asname = parts[2]
    try:
      module = importlib.import_module(module_name)
      globals()[asname] = module
    except Exception as e:
      print(e)
      print("{0} not installed\Trying to install it".format(module_name))
      try:
        !pip install $module_name
        module = importlib.import_module(module_name)  
        globals()[asname] = module        
      except Exception as e2:
        print(e2)
        fails.append(module_name)
  if fails:
    msg = "Not able to import or install\n{0}".format(fails)
  else:
    msg = "Success. imported {0}".format(imports)
  if verbosity > 0:
    print(msg)
  return fails
    
def is_defined(varnames=None, verbosity=0):
  if isinstance(varnames, str):
    varnames = [varnames]
  for v in varnames:
    try:
      eval(v)
      if verbosity > 0:
        print("'{0}'' is defined")
    except Exception as e:
      warnings.warn("trying to eval '{0}'".format(v))
      print(e)

imports = [ 'fsspec',
           'nltk',
            'pandas as pd',
            'numpy as np']
varnames = ['fsspec', 'pd']
global_imports(imports, verbosity=1)
# check if the imports worked
is_defined([x.split()[-1] for x in imports])

Success. imported ['fsspec', 'nltk', 'pandas as pd', 'numpy as np']


In [ ]:
import os
import sys
import datetime
import unicodedata
import warnings
import string
import re
import random
import json
import glob
import fsspec
from __future__ import unicode_literals, print_function, division
import re
# Not using torch for this initial part
if False:
  import torch
  import torch.nn as nn
  from torch import optim
  import torch.nn.functional as F
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# code to show what versions of modules I'm using
import inspect
mlist = list(filter(lambda x: inspect.ismodule(x[1]), locals().items()))
vi = sys.version_info
print("version {0}.{1}.{2} of Python".format(vi.major, vi.minor, vi.micro))
for name, mod in mlist:
    mname = name
    if name.startswith("__"):
      print(name, mod)
      continue
    if hasattr(mod, "__version__"):
        mname = name
        if hasattr(mod, "__path__"):
            mname = os.path.split(mod.__path__[0])[1]
        print("version {1} of {0} as {2} ".format(mname, name, mod.__version__))
    elif hasattr(mod, "__file__") and "site-packages" in mod.__file__:
        print("No __version__ for {0} as {1}".format(mname, name))
del mod
del name

version 3.6.9 of Python
__builtin__ <module 'builtins' (built-in)>
__builtins__ <module 'builtins' (built-in)>
version fsspec of fsspec as 0.8.4 
version nltk of nltk as 3.2.5 
version pd of pandas as 1.1.4 
version np of numpy as 1.18.5 
version re of re as 2.2.1 
version json of json as 2.0.9 


In [ ]:
if 'google.colab' in str(get_ipython()):
  print('Running on CoLab')
  # to mount google drive for the input files
  from google.colab import drive
  drive.mount('/content/gdrive', force_remount=False)  
  dpath = "/content/gdrive/My Drive/data/mlp_covid19/16119_db21c91a1ab47385bb13773ed8238c31"  
else:
  print('Not running on CoLab')
  dpath = "./data/16119_db21c91a1ab47385bb13773ed8238c31"

Running on CoLab
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# make a list of all the file paths in the data directory
jfiles = os.listdir(dpath)
jpaths = [os.path.join(dpath, jf) for jf in jfiles]
print("\n".join(sorted(jfiles[:4])))

16119_webhose_2020_03_db21c91a1ab47385bb13773ed8238c31_0000005.json
16119_webhose_2020_03_db21c91a1ab47385bb13773ed8238c31_0000015.json
16119_webhose_2020_03_db21c91a1ab47385bb13773ed8238c31_0000016.json
16119_webhose_2020_03_db21c91a1ab47385bb13773ed8238c31_0000023.json


In [ ]:
# only using 2 files
# 16119_webhose_2019_12_db21c91a1ab47385bb13773ed8238c31_0000001.json 
# 16119_webhose_2020_01_db21c91a1ab47385bb13773ed8238c31_0000001.json.

files_list = ["16119_webhose_2019_12_db21c91a1ab47385bb13773ed8238c31_0000001.json",
      "16119_webhose_2020_01_db21c91a1ab47385bb13773ed8238c31_0000001.json"]
#check that these are indeed in the directory
all_files = set(os.listdir(dpath))
for f in files_list:
    if f not in all_files:
        warnings.warn("{0} not found".format(f))
        continue
    fpath = os.path.join(dpath, f)
    if not os.path.isfile(fpath):
        warnings.warn("{0} found but not a file".format(f))
    fbytes = os.path.getsize(fpath)
    print("{0} {1}  MB  {2} GB".format(f, np.round(fbytes/10**6, 2),
                                       np.round(fbytes/10**9, 3)))

16119_webhose_2019_12_db21c91a1ab47385bb13773ed8238c31_0000001.json 0.56  MB  0.001 GB
16119_webhose_2020_01_db21c91a1ab47385bb13773ed8238c31_0000001.json 524.38  MB  0.524 GB


In [ ]:
# try reading the data in the first file using the json module
# raised error
f1 = files_list[0]
p1 = os.path.join(dpath, f1)
encoding = 'utf-8'
try:
  with open(p1, encoding='utf-8') as f:
    data = json.load(f)
except Exception as e:
  print(e)
  # sys.exc_info returns (type, value, traceback)
  extype, exval, tb = sys.exc_info()
  warnings.warn(str(extype))

Extra data: line 2 column 1 (char 8133)


line:13  cat:<UserWarning>  msg:<class 'json.decoder.JSONDecodeError'>


In [ ]:
# try reading the data with pandas
# raised error
f1 = files_list[0]
encoding = 'utf-8'
p1 = os.path.join(dpath, f1)
try:
  data = pd.read_json(p1, encoding=encoding)
except Exception as e:
  print(e)   
  # sys.exc_info returns (type, value, traceback)
  extype, exval, tb = sys.exc_info()
  warnings.warn(str(extype)) 

Trailing data


line:12  cat:<UserWarning>  msg:<class 'ValueError'>


In [ ]:
# So read in as text 
txt_data = ''
encoding = 'utf-8'
for f in files_list:
  fpath = os.path.join(dpath, f)
  print("reading {0} {1}".format(f, now()))
  with open(fpath, mode='r',encoding=encoding) as fp:
    new_data = fp.read()
  print("len {0} {1}".format(len(new_data), datetime.datetime.now()))
  txt_data += new_data

print("len of txt data {0}".format(len(txt_data)))
print(now())

reading 16119_webhose_2019_12_db21c91a1ab47385bb13773ed8238c31_0000001.json 2020-12-06 19:12:12.295694
len 561492 2020-12-06 19:12:12.338889
reading 16119_webhose_2020_01_db21c91a1ab47385bb13773ed8238c31_0000001.json 2020-12-06 19:12:12.339057
len 522019220 2020-12-06 19:12:18.863726
len of txt data 522580712
2020-12-06 19:12:19.613879


In [ ]:
# split into lines
# eval each line
# WARNING: should not do this with untrusted files
txt_lines = txt_data.split('\n')
print("{0} lines".format(len(txt_lines)))
dlist = []
for i, l in enumerate(txt_lines):
  try:
    d = eval(l) 
    dlist.append(d)
  except Exception as e:
    max_out = 30
    msg = "line{0}, {1}...{2} \n{3}".format(i,l[:20], l[-20:], e)
    warnings.warn(msg)
print("len dlist {0}".format(len(dlist)))

94402 lines


line:14  cat:<UserWarning>  msg:line103, {"organizations": []...highlightTitle": ""} 
invalid syntax (<string>, line 1)


len dlist 94401


In [ ]:
#extract text and title
dataset = [d['text'] for d in dlist]
target = [d['title'] for d in dlist]
# The length of the list dataset and target will be 190138.
print("dataset len= {0}, target len= {1}\n".format(len(dataset), len(target)))
names = ("dataset", "target")
n = 2
for name in names:
  print("first {0} from {1}".format(n,name))
  items = eval(name)[:n]
  maxlen = 30
  for i, item in enumerate(items):
    suffix = "..." if len(item) > maxlen else ""
    print("{0}: {1}{2}".format(i, item[:maxlen], suffix))
  print()

dataset len= 94401, target len= 94401

first 2 from dataset
0: Dublin, The “Swine Healthcare ...
1: FDA launches app for health ca...

first 2 from target
0: Global Swine Healthcare Market...
1: FDA launches app for health ca...



In [ ]:
contraction_map = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
                           "you're": "you are", "you've": "you have"}

In [ ]:
# took a litte trial and error to get that
# have to call nltk.download() first
nltk.download('stopwords')
stop_words = set(nltk.corpus.stopwords.words('english'))  
print(type(stop_words))
print(list(stop_words)[:4])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
<class 'set'>
['mightn', 'can', 'were', 'not']


In [ ]:
# my preprocess funtion
def preprocess(text, contraction_map= None, stop_words = None):
  if not contraction_map:
    contraction_map = {}
  if not stop_words:
    stop_words = []
  if not isinstance(stop_words, set):
    stop_words = set(stop_words)

  # Split the text using Python split() function
  toks = text.split()
  # Apply the contraction hashmap on all the words of the text
  toks = [contraction_map[x] if x in contraction_map.keys() else x for x in toks]  
  # Remove the stopwords that are in English
  toks = [x for x in toks if x not in stop_words]

  # now rejoin to apply other transforms on text string
  res = ' '.join(toks)
  # Convert text to lowercase
  res = res.lower()    
  # Remove 's. For example yours becomes your
  res = res.replace("'s",'') # convert your's -> your
  # Use regular expression to remove parentheses outside a word. For example (word) becomes word
  res = re.sub("\(|\)", "", res) 
  # Use regular expression to remove punctuations
  res = re.sub(r'[^a-zA-Z0-9. ]','',res)
  # Use regular expression to add a space character before and after the full stop. For example . becomes .
  res = re.sub("\.", " . ", res)
  return res
test = "The boys can't .(go) there!."
test_out = preprocess(test, 
           contraction_map=contraction_map, 
           stop_words=stop_words)
print("{0}\n{1}".format(test, test_out))

The boys can't .(go) there!.
the boys cannot  . go there . 


In [ ]:
# suggested preprocess function
def preprocess_ref(text, contraction_map, stop_words):
    text = text.lower() # lowercase
    text = text.split() # convert have'nt -> have not
    for i in range(len(text)):
        word = text[i]
        if word in contraction_map:
            text[i] = contraction_map[word]
    text = " ".join(text)
    text = text.split()
    newtext = []
    for word in text:
        if word not in stop_words:
            newtext.append(word)
    text = " ".join(newtext)
    text = text.replace("'s",'') # convert your's -> your
    text = re.sub(r'\(.*\)','',text) # remove (words)
    text = re.sub(r'[^a-zA-Z0-9. ]','',text) # remove punctuations
    text = re.sub(r'\.',' . ',text)
    return text
test = "The boys can't .(go) there!."
test_out = preprocess(test, 
           contraction_map=contraction_map, 
           stop_words=stop_words)
print("{0}\n{1}".format(test, test_out))    

The boys can't .(go) there!.
the boys cannot  . go there . 


In [ ]:
%time X = [preprocess(line, contraction_map=contraction_map, stop_words=stop_words) for line in dataset]

CPU times: user 22 s, sys: 125 ms, total: 22.1 s
Wall time: 22.1 s


In [ ]:
%time X_ref = [preprocess_ref(line, contraction_map=contraction_map, stop_words=stop_words) for line in dataset]

CPU times: user 24.1 s, sys: 133 ms, total: 24.2 s
Wall time: 24.2 s


In [ ]:
%time Y = [preprocess(line, contraction_map=contraction_map, stop_words=stop_words) for line in target]

CPU times: user 829 ms, sys: 7.85 ms, total: 836 ms
Wall time: 836 ms


In [ ]:
%time Y_ref = [preprocess_ref(line, contraction_map=contraction_map, stop_words=stop_words) for line in target]

CPU times: user 855 ms, sys: 32.8 ms, total: 888 ms
Wall time: 887 ms


In [ ]:
# At this point, the length of X and Y should both be 190138.
print("len X= {0}, len Y= {1}".format(len(X), len(Y)))
print("len X_ref= {0}, len Y_ref= {1}".format(len(X_ref), len(Y_ref)))
#assert(len(x) == 190138)

len X= 94401, len Y= 94401
len X_ref= 94401, len Y_ref= 94401


In [ ]:
# How does this relate to the delivarble?
short_text=[]
short_summary=[]
max_len_text = 600
max_len_target = 30
for i in range(len(dataset)):
    if(len(target[i].split())<=max_len_target and len(dataset[i].split())<=max_len_text):
        short_text.append(dataset[i])
        short_summary.append(target[i])

temp_df=pd.DataFrame({'text':short_text,'summary':short_summary})
print("temp_df shape {0}".format(temp_df.shape))
# len(temp_df) being 130736.

temp_df shape (64892, 2)


In [ ]:
temp_df.head(2)

,text,summary
0,FDA launches app for health care professionals...,FDA launches app for health care professionals...
1,"Of all of Regina Yan ’s many traits, an open m...",C-Suite Awards: Regina Yan


In [ ]:
# remove empty strings from summary and the text column.
newdf = temp_df[temp_df['summary'].str.strip().astype(bool)]
df = newdf[newdf['text'].str.strip().astype(bool)]
print("df shape {0}".format(df.shape))
print(df.head(3))

df shape (62357, 2)
                                                text                                            summary
0  FDA launches app for health care professionals...  FDA launches app for health care professionals...
1  Of all of Regina Yan ’s many traits, an open m...                         C-Suite Awards: Regina Yan
2  The CURE ID app allows clinicians to share and...  FDA Launches Infectious Disease Crowdsourcing ...


In [ ]:
SOS_token = 0
EOS_token = 1
MAX_LENGTH = 20

SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [ ]:
df.head(2)
X = list(df['text'].values)
Y = list(df['summary'].values)

In [ ]:
def readData(text, summary):
  print("Reading lines...")
  input_lang = Lang(text)
  output_lang = Lang(summary)
  pairs = [[text[i],summary[i]] for i in range(len(text))]
  return (input_lang, output_lang, pairs)

def prepareData(tlist, slist):
  input_lang, output_lang, pairs = readData(tlist, slist)
  print("Read %s sentence pairs" % len(pairs))
  print("Counting words...")  
  [input_lang.addSentence(t) for t in tlist]
  [output_lang.addSentence(s) for s in slist] 
  return input_lang, output_lang, pairs 

In [ ]:
input, output, pairs = prepareData(X,Y)

Reading lines...
Read 62357 sentence pairs
Counting words...
